# STAC Catalog 

Invoke a Common Workflow Language CommandLineTool to generate the STAC Catalog

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac



## Setup

In [16]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC Catalog generation CommandLineTool

Inspect and use `cwltool` to run the STAC Catalog generation definition:


In [17]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][6]' -

class: CommandLineTool
id: stac-collection
requirements:
  InlineJavascriptRequirement: {}
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: ghcr.io/eoap/zarr-cloud-native-format/stac-collection@sha256:a6ab00463338b4069eb57b3944d92cf39fd2307176ad5b5cb357836c56a22ca2
baseCommand: ["stac-collection"]
arguments: []
inputs:
  item:
    type:
      type: array
      items: string
      inputBinding:
        prefix: --input-item
  rasters:
    type:
      type: array
      items: File
      inputBinding:
        prefix: --otsu
  ndwis:
    type:
      type: array
      items: File
      inputBinding:
        prefix: --ndwi
outputs:
  temp_stac_catalog:
    label: temporary STAC catalog with COG outputs
    outputBinding:
      glob: .
    type: Directory


Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [18]:
cat water-bodies-results.json 

{
    "detected_water_body": {
        "location": "file:///workspace/zarr-cloud-native-format/runs/otsu.tif",
        "basename": "otsu.tif",
        "class": "File",
        "checksum": "sha1$a7f9a22f096d7bb5e2a0ecfe0abba2eab2350f1f",
        "size": 1100785,
        "path": "/workspace/zarr-cloud-native-format/runs/otsu.tif"
    },
    "ndwi": {
        "location": "file:///workspace/zarr-cloud-native-format/runs/norm_diff.tif",
        "basename": "norm_diff.tif",
        "class": "File",
        "checksum": "sha1$9e77047b2de97a3c2bf78e18e29867463c418c53",
        "size": 233064610,
        "path": "/workspace/zarr-cloud-native-format/runs/norm_diff.tif"
    }
}


Let's build the job parameters file with the otsu.tif file and the associated STAC Item input

In [26]:
ostu_tif=$(cat water-bodies-results.json  | jq '.detected_water_body.path')
ndwi_tif=$(cat water-bodies-results.json  | jq '.ndwi.path')
echo ${ostu_tif}
echo ${ndwi_tif}

"/workspace/zarr-cloud-native-format/runs/otsu.tif"
"/workspace/zarr-cloud-native-format/runs/norm_diff.tif"


In [20]:
item=$( cat convert-search-results.json | jq '.items[0]' )

echo ${item}

"https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"


In [27]:
cat <<EOF > stac-generation-params.yaml
item: 
- ${item}
rasters:
- class: File
  path: ${ostu_tif}
ndwis:
- class: File
  path: ${ndwi_tif}
EOF

cat stac-generation-params.yaml | yq .

item:
  - "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_10TFK_20210728_0_L2A"
rasters:
  - class: File
    path: "/workspace/zarr-cloud-native-format/runs/otsu.tif"
ndwis:
  - class: File
    path: "/workspace/zarr-cloud-native-format/runs/norm_diff.tif"


In [29]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac-collection \
    stac-generation-params.yaml > stac-generation-results.json 2> stac-generation.log

Let's look at the content of the stderr:

In [ ]:
cat stac-generation.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac'
INFO [job stac] /tmp/m9tf6uoo$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/m9tf6uoo,target=/oAmJOc \
    --mount=type=bind,source=/tmp/9est9nzk,target=/tmp \
    --mount=type=bind,source=/workspace/zarr-cloud-native-format/runs/otsu.tif,target=/var/lib/cwl/stg0dfb665e-3b44-48b1-943e-f38b4ff6d03c/otsu.tif,readonly \
    --workdir=/oAmJOc \
    --read-only=true \
    --user=1000:1000 \
    --rm \
    --cidfile=/tmp/qowl65pu/20260114122904-265269.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/oAmJOc \
    --env=PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin \
    --env=PYTHONPATH=/app \
    stac:bb \
    python \
    -m \
    app \
    --input-item \
    https://earth-search.aws.element84.com/v1/colle

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [30]:
cat stac-generation-results.json | jq . -

{
  "temp_stac_catalog": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc",
    "basename": "hicmgmcc",
    "class": "Directory",
    "listing": [
      {
        "class": "Directory",
        "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies",
        "basename": "water-bodies",
        "listing": [
          {
            "class": "File",
            "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies/collection.json",
            "basename": "collection.json",
            "size": 1351,
            "checksum": "sha1$299a72c299d255d14b7655e76824d9119a72cc6e",
            "path": "/workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies/collection.json"
          },
          {
            "class": "Directory",
            "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies/S2A_10TFK_20210728_0_L2A",
            "basename": "S2A_10TFK_20210728_0_L2A",
        